In [1]:
# !pip install transformers
print("okay")

okay


### This is the training file for non paired experiment (baseline)

In [2]:
import pandas as pd
import os, time
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
tf.get_logger().setLevel('ERROR')
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import BertForSequenceClassification, BertTokenizerFast, LongformerForSequenceClassification

# specify GPU
device = torch.device("cuda")

C:\Users\YunTian\anaconda3\envs\ampligraph\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Seed
import random
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

In [5]:
train_df = pd.read_csv("./final_train.csv")
train_label_df = pd.read_csv("./final_train_labels.csv")
train_df = train_df[['query', 'p1', 'p2', 'p3']]
train_label_df = train_label_df[['label']]

val_df = pd.read_csv("./final_val_text.csv")
val_label_df = pd.read_csv("./final_val_labels.csv")
val_df = val_df[['query', 'p1', 'p2', 'p3']]
val_label_df = val_label_df[['label']]

test_df = pd.read_csv("./final_test_text.csv")
test_label_df = pd.read_csv("./final_test_labels.csv")
test_df = test_df[['query', 'p1', 'p2', 'p3']]
test_label_df = test_label_df[['label']]


df_train = pd.concat([train_df, train_label_df], axis=1)
df_val = pd.concat([val_df, val_label_df], axis=1)
df_test = pd.concat([test_df, test_label_df], axis=1)


In [6]:
from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [7]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer

# data set generator for single query
class TrainingDataBert(Dataset):
    def __init__(self, train_df, val_df, tokenizer):
        print("Loading Datasets")
        self.train_df = train_df
        self.val_df = val_df

        self.rejected = 0
        self.tokenizer = tokenizer

        self.train_data = None
        self.val_data = None
        self.count = 0
        self.init_data()
        print("Datasets loaded")
        
    def init_data(self):
        print("-"*8+" Creating data  "+"-"*8)
        self.train_data = self.load_data(self.train_df)
        print("-"*8+" Train data created "+"-"*8+"\n")
        self.val_data = self.load_data(self.val_df)
        print("-"*8+" Val data created "+"-"*8+"\n")
        
    def load_data(self, df):
        def create_ids_and_mask(premise, hypothesis, label):
            # outputs = self.tokenizer([hypothesis, premise], add_special_tokens=True, max_length = 512, pad_to_max_length=True, truncation=True,)
            outputs = self.tokenizer([premise], add_special_tokens=True, max_length = 512, pad_to_max_length=True, truncation=True,)
            attention_mask_ids = outputs['attention_mask'][0]
            token_ids = outputs['input_ids'][0]
            
            final_token_ids.append(token_ids)
            final_mask_ids.append(attention_mask_ids)
            final_y.append(label)
            
        MAX_LEN = 512 - 3
        final_token_ids = []
        final_mask_ids = []
        final_y = []
    
        premise_list = df['query'].to_list()
        hypothesis1_list = df['p1'].to_list()
        hypothesis2_list = df['p2'].to_list()
        hypothesis3_list = df['p3'].to_list()
        label_list = df['label'].to_list()
        
        for (premise, p1, p2, p3, label) in zip(premise_list, hypothesis1_list, hypothesis2_list, hypothesis3_list, label_list):
            len1 = len(premise) + len(p1)
            len2 = len(premise)+ len(p2)
            len3 = len(premise)+ len(p3)
            if (len1 <= MAX_LEN and len2 <= MAX_LEN and len3 <= MAX_LEN):
                create_ids_and_mask(premise=p1, hypothesis=premise, label=label)
                self.count += 1
            else:
                self.rejected += 1
    
        final_token_ids = torch.tensor(final_token_ids)
        final_mask_ids = torch.tensor(final_mask_ids)
        final_y = torch.tensor(final_y)

        dataset = TensorDataset(final_token_ids, final_mask_ids, final_y)
        print("Rejected:",self.rejected)
        print("Dataset size:", df.shape[0]-self.rejected)
        return dataset
    
    def get_data_loaders(self, bs, shuffle=True):
        print("Generating dataloader")
        train_loader = DataLoader(
          self.train_data,
          shuffle=shuffle,
          batch_size=bs,
        )

        val_loader = DataLoader(
          self.val_data,
          shuffle=shuffle,
          batch_size=bs,
        )
        print("Generated dataloaders")
        return train_loader, val_loader

dataset = TrainingDataBert(df_train, df_val, tokenizer)
train_dataset, val_dataset = dataset.get_data_loaders(bs=7)

Loading Datasets
-------- Creating data  --------


C:\Users\YunTian\anaconda3\envs\ampligraph\lib\site-packages\transformers\tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Rejected: 2075
Dataset size: 15210
-------- Train data created --------

Rejected: 2473
Dataset size: 1231
-------- Val data created --------

Datasets loaded
Generating dataloader
Generated dataloaders


In [8]:
from torch.optim import Adam
import time

num_epochs = 10
model.to(device)
optimizer = Adam(model.parameters(), lr=5e-5)

def multi_acc(y_pred, y_test):
    acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
    return acc


def train(model, train_loader, val_loader, optimizer):
  print("-"*8+" start training "+"*"*8)
  best_valid_loss = float('inf')
  total_step = len(train_loader)
    
  for epoch in range(num_epochs):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    
    print("Training phase:",epoch)
    for batch_idx, batch in enumerate(train_loader):
      optimizer.zero_grad()
      torch.cuda.empty_cache()
      
      if batch_idx % 100 == 0 and not batch_idx == 0:
          print('  Batch {:>5,}  of  {:>5,}.'.format(batch_idx, len(train_loader)))
          print("Avg Loss:",str(total_train_loss/batch_idx), " - ", "Avg Acc:",str(total_train_acc/batch_idx))
        
      text_ids, mask_ids, y = [r.to(device) for r in batch]

      train_loss, prediction = model(input_ids=text_ids, attention_mask=mask_ids, labels=y).values()

      train_acc = multi_acc(prediction, y)
      train_loss.backward()
      optimizer.step()
      # lr_scheduler.step()
       
      total_train_loss += train_loss.item()
      total_train_acc  += train_acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    
    print("Evaluation phase:",epoch)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, batch in enumerate(val_loader):
        torch.cuda.empty_cache()
        optimizer.zero_grad()
        if batch_idx % 100 == 0 and not batch_idx == 0:
          print('  Batch {:>5,}  of  {:>5,}.'.format(batch_idx, len(val_loader)))
        
        text_ids, mask_ids, y = [r.to(device) for r in batch]

        loss, prediction = model(input_ids=text_ids,
                             attention_mask=mask_ids, 
                             labels=y).values()
        
        acc = multi_acc(prediction, y)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()
    
    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    end = time.time()
    
    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        torch.save(model.state_dict(), 'bert_single_query_new.pt')
    
    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("Time taken:",str(start-end))

In [ ]:
train(model, train_dataset, val_dataset, optimizer)

-------- start training ********
Training phase: 0
  Batch   100  of  2,173.
Avg Loss: 0.6020444223284721  -  Avg Acc: 0.6757143187522888
  Batch   200  of  2,173.
Avg Loss: 0.5894287674501538  -  Avg Acc: 0.6842857470363378
  Batch   300  of  2,173.
Avg Loss: 0.5798048480103414  -  Avg Acc: 0.6957143196463584
  Batch   400  of  2,173.
Avg Loss: 0.5719570819102228  -  Avg Acc: 0.7039286049455404
  Batch   500  of  2,173.
Avg Loss: 0.5610306170135736  -  Avg Acc: 0.7097143183350563
  Batch   600  of  2,173.
Avg Loss: 0.5556943279132247  -  Avg Acc: 0.7176190807422003
  Batch   700  of  2,173.
Avg Loss: 0.5536616097816399  -  Avg Acc: 0.720612277175699
  Batch   800  of  2,173.
Avg Loss: 0.5462415262684226  -  Avg Acc: 0.7258928895369172
  Batch   900  of  2,173.
Avg Loss: 0.5424625481168429  -  Avg Acc: 0.7290476511253251
  Batch 1,000  of  2,173.
Avg Loss: 0.5384855469167232  -  Avg Acc: 0.7308571749031544
  Batch 1,100  of  2,173.
Avg Loss: 0.5338805542547594  -  Avg Acc: 0.7340260062

  Batch   400  of    473.
Epoch 4: train_loss: 0.3210 train_acc: 0.8752 | val_loss: 0.6381 val_acc: 0.7021
Time taken: -693.8249332904816
Training phase: 4
  Batch   100  of  2,173.
Avg Loss: 0.6763729751110077  -  Avg Acc: 0.6200000289082527
  Batch   200  of  2,173.
Avg Loss: 0.6708044067025185  -  Avg Acc: 0.6214285983890295
  Batch   300  of  2,173.
Avg Loss: 0.6683305434385935  -  Avg Acc: 0.6242857425908248
  Batch   400  of  2,173.
Avg Loss: 0.671298850402236  -  Avg Acc: 0.6203571715205908
  Batch   500  of  2,173.
Avg Loss: 0.6313201570510865  -  Avg Acc: 0.6551428859233857
  Batch   600  of  2,173.
Avg Loss: 0.6350631176928679  -  Avg Acc: 0.6547619339327018
  Batch   700  of  2,173.
Avg Loss: 0.639342508656638  -  Avg Acc: 0.6516326823404857
  Batch   800  of  2,173.
Avg Loss: 0.6407329254969955  -  Avg Acc: 0.6510714576207102
  Batch   900  of  2,173.
Avg Loss: 0.6454563433925311  -  Avg Acc: 0.6431746324400107
  Batch 1,000  of  2,173.
Avg Loss: 0.6490201849341393  -  Avg 

In [ ]:
dataset = TrainingDataBert(df_train, df_val, tokenizer)
train_dataset, val_dataset = dataset.get_data_loaders(bs=7)